In [0]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
import os
os.chdir('/gdrive/My Drive/main_data')

# Let's do this man!

In [0]:
import math, os
import numpy as np
import keras
from keras.layers import Dropout, Flatten, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.preprocessing import image


PATH = os.getcwd()
# Define data path
DATASET_DIR = PATH + '/train'

SIZE = (224, 224)
BATCH_SIZE = 16
VALIDATION_SPLIT = 0.3
IMG_SIZE = ( 224, 224, 3)

In [0]:
# How many data and classes ..
num_dataset_samples = sum([len(files) for r, d, files in os.walk(DATASET_DIR)])

np.random.seed(42)
num_train_steps = math.floor((num_dataset_samples*(1-VALIDATION_SPLIT))/BATCH_SIZE)
num_valid_steps = math.floor((num_dataset_samples*VALIDATION_SPLIT)/BATCH_SIZE)

generator = image.ImageDataGenerator(validation_split=VALIDATION_SPLIT)

batches = generator.flow_from_directory(DATASET_DIR, subset='training', target_size=SIZE, class_mode='categorical', shuffle=True, batch_size=BATCH_SIZE)
val_batches = generator.flow_from_directory(DATASET_DIR, subset='validation', target_size=SIZE, class_mode='categorical', shuffle=True, batch_size=BATCH_SIZE)


Found 996 images belonging to 7 classes.
Found 424 images belonging to 7 classes.


In [0]:

#  Let's use mobilenet as it is simple and needs less memory
# model = keras.applications.mobilenet.MobileNet(input_shape=(224, 224, 3),  include_top=False, weights='imagenet')
# model = keras.applications.mobilenet_v2.MobileNetV2(input_shape=(224, 224, 3),  include_top=False, weights='imagenet')
model = keras.applications.nasnet.NASNetMobile(input_shape=(224, 224, 3),  include_top=False, weights='imagenet')

W0623 16:58:23.955283 140064366757760 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0623 16:58:24.163242 140064366757760 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3980: The name tf.nn.avg_pool is deprecated. Please use tf.nn.avg_pool2d instead.



19996672/19993432 [==============================] - 1s 0us/step


In [0]:
# classes
classes = list(iter(batches.class_indices))
classes

['bodha',
 'golden_temple',
 'nyatapola',
 'pashupati_temple',
 'patan',
 'rara_lake',
 'tilicho_lake']

In [0]:

# Our Model starts
x = Sequential()
x.add(model)
x.add(Flatten())
x.add(Dense(28, activation='relu'))
x.add(Dropout(0.25))
x.add(BatchNormalization())
x.add(Dense(len(classes), activation="softmax"))


In [0]:

# Compile
x.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


In [0]:

# logs on csv file... 
for c in batches.class_indices:
    classes[batches.class_indices[c]] = c
x.classes = classes
filename='model_train_new.csv'
csv_log=keras.callbacks.CSVLogger(filename, separator=',', append=False)


In [0]:
# Early stopping if loss doesnt improve
early_stopping = EarlyStopping(patience=10)
checkpointer = ModelCheckpoint('nasnet_best.h5', verbose=1, save_best_only=True)

tensorboard_callback = keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=BATCH_SIZE, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)

# Fit and save
his = x.fit_generator(batches, steps_per_epoch=num_train_steps, epochs=20, callbacks=[early_stopping, checkpointer, tensorboard_callback, csv_log], validation_data=val_batches, validation_steps=num_valid_steps)
x.save('nasnet_final.h5')

scoreSeg = x.evaluate_generator(val_batches, num_dataset_samples*VALIDATION_SPLIT )
print("Accuracy = ",scoreSeg[1])

Epoch 1/20
62/62 [==============================] - 60s 972ms/step - loss: 1.0477 - acc: 0.6483 - val_loss: 0.4641 - val_acc: 0.8873

Epoch 00001: val_loss improved from inf to 0.46406, saving model to nasnet_best.h5
Epoch 2/20
62/62 [==============================] - 24s 382ms/step - loss: 0.4646 - acc: 0.9002 - val_loss: 0.3097 - val_acc: 0.9387

Epoch 00002: val_loss improved from 0.46406 to 0.30970, saving model to nasnet_best.h5
Epoch 3/20
62/62 [==============================] - 24s 392ms/step - loss: 0.3392 - acc: 0.9436 - val_loss: 0.3319 - val_acc: 0.9387

Epoch 00003: val_loss did not improve from 0.30970
Epoch 4/20
62/62 [==============================] - 24s 392ms/step - loss: 0.2625 - acc: 0.9698 - val_loss: 0.2654 - val_acc: 0.9387

Epoch 00004: val_loss improved from 0.30970 to 0.26537, saving model to nasnet_best.h5
Epoch 5/20
62/62 [==============================] - 24s 391ms/step - loss: 0.2201 - acc: 0.9728 - val_loss: 0.2591 - val_acc: 0.9436

Epoch 00005: val_loss 

In [0]:
# x.summary()

In [0]:
!pip install tensorflow==1.13.0rc1


     |████████████████████████████████| 92.6MB 1.2MB/s 
     |████████████████████████████████| 3.1MB 34.6MB/s 
     |████████████████████████████████| 368kB 26.8MB/s 
  Found existing installation: tensorboard 1.13.1
    Uninstalling tensorboard-1.13.1:
      Successfully uninstalled tensorboard-1.13.1
  Found existing installation: tensorflow-estimator 1.14.0rc1
    Uninstalling tensorflow-estimator-1.14.0rc1:
      Successfully uninstalled tensorflow-estimator-1.14.0rc1
  Found existing installation: tensorflow 1.14.0rc1
    Uninstalling tensorflow-1.14.0rc1:
      Successfully uninstalled tensorflow-1.14.0rc1


In [0]:
from tensorflow import lite
converter = lite.TFLiteConverter.from_keras_model_file( '/gdrive/My Drive/main_data/mobi_best.h5') 
tfmodel = converter.convert() 
converter.quantize = False

open ('/gdrive/My Drive/main_data/model.tflite' , "wb") .write(tfmodel)


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph
INFO:tensorflow:Froze 268 variables.
INFO:tensorflow:Converted 268 variables to const ops.


15878708